In [1]:
import pandas as pd

In [146]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday
from pandas.tseries.offsets import DateOffset, Day
import datetime
import numpy as np


### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [1]:
# пример требуемого DataFrame 
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

Считываем календарь. Возьмём только месяцы и переведём их в числовой формат

In [438]:
df1 = pd.read_csv('./data/data-20191112T1252-structure-20191112T1247.csv', names=[i for i in range(18)], usecols=[i for i in range(13)], index_col=0, header=0)
df1.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
1999,"1,2,3,4,6*,7,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3,4,8,9,10,15,16,22,23,29,30","5,6,11*,12,13,14,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,8,13,14,20,21,27,28","4,5,11,12,13,18,19,25,26,31*"
2000,"1,2,3,4,6*,7,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","4,5,7*,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","1,2,6,7,8*,9,13,14,20,21,27,28","3,4,10,11,12,17,18,24,25","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,7,11,12,18,19,25,26","2,3,9,10,11*,12,16,17,23,24,30,31"


Сейчас каждое значение датафрейма имеет тип строки, переведём его в список

In [439]:
df2 = df1.applymap(lambda x: str(x).split(','))
df2.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
1999,"[1, 2, 3, 4, 6*, 7, 9, 10, 16, 17, 23, 24, 30,...","[6, 7, 13, 14, 20, 21, 27, 28]","[6, 7, 8, 13, 14, 20, 21, 27, 28]","[3, 4, 10, 11, 17, 18, 24, 25, 30*]","[1, 2, 3, 4, 8, 9, 10, 15, 16, 22, 23, 29, 30]","[5, 6, 11*, 12, 13, 14, 19, 20, 26, 27]","[3, 4, 10, 11, 17, 18, 24, 25, 31]","[1, 7, 8, 14, 15, 21, 22, 28, 29]","[4, 5, 11, 12, 18, 19, 25, 26]","[2, 3, 9, 10, 16, 17, 23, 24, 30, 31]","[6, 7, 8, 13, 14, 20, 21, 27, 28]","[4, 5, 11, 12, 13, 18, 19, 25, 26, 31*]"
2000,"[1, 2, 3, 4, 6*, 7, 8, 9, 15, 16, 22, 23, 29, 30]","[5, 6, 12, 13, 19, 20, 26, 27]","[4, 5, 7*, 8, 11, 12, 18, 19, 25, 26]","[1, 2, 8, 9, 15, 16, 22, 23, 29, 30]","[1, 2, 6, 7, 8*, 9, 13, 14, 20, 21, 27, 28]","[3, 4, 10, 11, 12, 17, 18, 24, 25]","[1, 2, 8, 9, 15, 16, 22, 23, 29, 30]","[5, 6, 12, 13, 19, 20, 26, 27]","[2, 3, 9, 10, 16, 17, 23, 24, 30]","[1, 7, 8, 14, 15, 21, 22, 28, 29]","[4, 5, 7, 11, 12, 18, 19, 25, 26]","[2, 3, 9, 10, 11*, 12, 16, 17, 23, 24, 30, 31]"


Создадим список выходных и праздников

In [440]:
russian_rules = [Holiday(name='Выходной или праздник', year=year, month=month, day= int(day.partition('+')[0]))
for year in df2.index for month in df2.columns for day in df2.loc[year][month] if day.find('*') == -1]

Создаём список предновогодних выходных (он пригодится в календаре)

In [441]:
holiday_before_new_year = [pd.to_datetime(datetime.datetime(year, 12, 31))
                           for year in df2.index if df2.loc[year][12][-1].find('*') == -1]
print("Список предновогодних выходных")
holiday_before_new_year

Список предновогодних выходных


[Timestamp('2000-12-31 00:00:00'),
 Timestamp('2001-12-31 00:00:00'),
 Timestamp('2005-12-31 00:00:00'),
 Timestamp('2006-12-31 00:00:00'),
 Timestamp('2007-12-31 00:00:00'),
 Timestamp('2011-12-31 00:00:00'),
 Timestamp('2012-12-31 00:00:00'),
 Timestamp('2014-12-31 00:00:00'),
 Timestamp('2016-12-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2018-12-31 00:00:00'),
 Timestamp('2022-12-31 00:00:00'),
 Timestamp('2023-12-31 00:00:00')]

Создадим свой календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`

In [443]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
    rules = russian_rules
    
russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', 
                                              calendar=RussianBusinessCalendar())

Создадим пустой one-hot DataFrame, который будем заполнять

In [444]:
data_range_ = pd.date_range(start='1999-01-01', end='2025-12-31', freq='D')
months_ = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']
week_days_ = ['пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс'] 
columns_ = ['выходной', 'праздник', 'предпраздничный рабочий день', 'последний день месяца', 'последний рабочий день месяца', 'Предновогодний выходной день'] + list(range(1, 13))+list(range(13, 20))
zeros_ = np.zeros((len(data_range_), len(columns_)), dtype=int)
calendar_df = pd.DataFrame(zeros_, columns=columns_, index=data_range_)
calendar_df

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
1999-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-12-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-12-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-12-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Заполняем последний день месяца, дни недели и месяцы

In [445]:
calendar_df.loc[calendar_df.index.is_month_end == True, 'последний день месяца'] = 1
for i in range(1, 13):
    calendar_df.loc[calendar_df.index.month == i, i] = 1
calendar_df.rename(columns={i+1: months_[i] for i in range(12)}, inplace=True)
for i in range(7):
    calendar_df.loc[calendar_df.index.dayofweek == i, i+13] = 1
calendar_df.rename(columns={i+13: week_days_[i] for i in range(7)}, inplace=True)

In [446]:
calendar_df

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
1999-01-01,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1999-01-02,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1999-01-03,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1999-01-04,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1999-01-05,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2025-12-28,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2025-12-29,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2025-12-30,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


Заполняем список предновогодних выходных

In [447]:
for date_ in holiday_before_new_year:
    calendar_df.loc[calendar_df.index == date_ , 'Предновогодний выходной день'] = 1

Заполняем список выходных

In [449]:
for day in russian_rules:
    calendar_df.loc[calendar_df.index == pd.to_datetime(datetime.datetime(day.year, day.month, day.day)), 'выходной'] = 1

Заполняем список праздников (так как в календаре не разделяют праздники и выходные, то я просто обозначил все государственные праздники)

In [467]:
calendar_df[calendar_df.index.]

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
1999-01-01,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1999-01-02,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1999-01-03,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1999-01-04,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1999-01-05,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1999-02-01,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1999-02-02,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1999-02-03,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1999-02-04,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1999-02-05,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


Заполняем список последних рабочих дней месяца

In [397]:
 pd.date_range('1/1/2014', periods=12, freq=)

DatetimeIndex(['2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30',
               '2014-05-30', '2014-06-30', '2014-07-31', '2014-08-29',
               '2014-09-30', '2014-10-31', '2014-11-28', '2014-12-31'],
              dtype='datetime64[ns]', freq='BM')

In [398]:
a = (calendar_df.index + russian_busday.BMonthEnd(1)).day

AttributeError: 'CustomBusinessDay' object has no attribute 'BMonthEnd'

In [396]:
ts.asfreq(russian_busday).index.

ValueError: Custom business days is not supported by is_year_start

In [406]:
from pandas.tseries import offset
from offset import BMonthEnd

ImportError: cannot import name 'offset' from 'pandas.tseries' (C:\Users\rinat\anaconda3\lib\site-packages\pandas\tseries\__init__.py)

1999

In [363]:
calendar_df.tail(2)

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
2025-12-30,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2025-12-31,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [382]:
calendar_df.loc[pd.to_datetime(datetime.datetime(2024, 12, 31))]

выходной                         0.0
праздник                         0.0
предпраздничный рабочий день     0.0
последний день месяца            1.0
последний рабочий день месяца    0.0
Предновогодний выходной день     0.0
январь                           0.0
февраль                          0.0
март                             0.0
апрель                           0.0
май                              0.0
июнь                             0.0
июль                             0.0
август                           0.0
сентябрь                         0.0
октябрь                          0.0
ноябрь                           0.0
декабрь                          1.0
пн                               0.0
вт                               1.0
ср                               0.0
чт                               0.0
пт                               0.0
сб                               0.0
вс                               0.0
январь                           NaN
февраль                          NaN
м

In [339]:
calendar_df.tail(1)

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
2025-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [317]:
df2[12]

0
1999            [4, 5, 11, 12, 13, 18, 19, 25, 26, 31*]
2000     [2, 3, 9, 10, 11*, 12, 16, 17, 23, 24, 30, 31]
2001           [1, 2, 8, 9, 12, 15, 16, 22, 23, 29, 30]
2002    [1, 7, 8, 11*, 12, 13, 14, 21, 22, 28, 29, 31*]
2003       [6, 7, 11*, 12, 13, 14, 20, 21, 27, 28, 31*]
2004            [4, 5, 11, 12, 13, 18, 19, 25, 26, 31*]
2005                 [3, 4, 10, 11, 17, 18, 24, 25, 31]
2006              [2, 3, 9, 10, 16, 17, 23, 24, 30, 31]
2007          [1, 2, 8, 9, 15, 16, 22, 23, 29*, 30, 31]
2008                [6, 7, 13, 14, 20, 21, 27, 28, 31*]
2009                [5, 6, 12, 13, 19, 20, 26, 27, 31*]
2010                [4, 5, 11, 12, 18, 19, 25, 26, 31*]
2011                 [3, 4, 10, 11, 17, 18, 24, 25, 31]
2012          [1, 2, 8, 9, 15, 16, 22, 23, 29*, 30, 31]
2013             [1, 7, 8, 14, 15, 21, 22, 28, 29, 31*]
2014                     [6, 7, 13, 14, 20, 21, 27, 28]
2015                [5, 6, 12, 13, 19, 20, 26, 27, 31*]
2016                 [3, 4, 10, 11, 17, 18, 24

In [294]:
calendar_df.loc[calendar_df.index.month == 12, 12] = 1
calendar_df.loc['2025-11-29']

выходной                         0.0
праздник                         0.0
предпраздничный рабочий день     0.0
последний день месяца            0.0
последний рабочий день месяца    0.0
пн                               0.0
вт                               0.0
ср                               0.0
чт                               0.0
пт                               0.0
сб                               0.0
вс                               0.0
Предновогодний выходной день     0.0
январь                           0.0
февраль                          0.0
март                             0.0
апрель                           0.0
май                              0.0
июнь                             0.0
июль                             0.0
август                           0.0
сентябрь                         0.0
октябрь                          0.0
ноябрь                           1.0
декабрь                          0.0
12                               NaN
Name: 2025-11-29 00:00:00, dtype: floa

In [297]:
calendar_df.rename(columns={i+1: months_[i] for i in range(12)}, inplace=True)

calendar_df.index.dayofweek

Int64Index([4, 5, 6, 0, 1, 2, 3, 4, 5, 6,
            ...
            0, 1, 2, 3, 4, 5, 6, 0, 1, 2],
           dtype='int64', length=9862)

In [305]:
calendar_df.head(10)

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
1999-01-01,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1999-01-02,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1999-01-03,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1999-01-04,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1999-01-05,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1999-01-06,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1999-01-07,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1999-01-08,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1999-01-09,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1999-01-10,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [254]:
# for i in range(1, 12):
#     print(calendar_df.loc[calendar_df.index.month == i, i])

In [248]:
calendar_df.loc['2025-12-31']

выходной                         0
праздник                         0
предпраздничный рабочий день     0
последний день месяца            1
последний рабочий день месяца    0
пн                               0
вт                               0
ср                               1
чт                               0
пт                               0
сб                               0
вс                               0
Предновогодний выходной день     0
1                                0
2                                0
3                                0
4                                0
5                                0
6                                0
7                                0
8                                0
9                                0
10                               0
11                               0
12                               0
Name: 2025-12-31 00:00:00, dtype: int32